In [2]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim

In [3]:
# CSVファイルの読み込み
df1 = pd.read_csv('xy(1).csv')
df2 = pd.read_csv('xy(2).csv')
df3 = pd.read_csv('xy(3).csv')
df4 = pd.read_csv('xy1.csv')
df5 = pd.read_csv('xy2.csv')
df6 = pd.read_csv('xy3.csv')

# データの抽出
x_data_1 = df1['x'].values
x_data_2 = df2['x'].values
x_data_3 = df3['x'].values
x_data_4 = df4['x'].values
x_data_5 = df5['x'].values
x_data_6 = df6['x'].values
y_data_1 = df1['y'].values
y_data_2 = df2['y'].values
y_data_3 = df3['y'].values
y_data_4 = df4['y'].values
y_data_5 = df5['y'].values
y_data_6 = df6['y'].values

# データにラベルを付ける
labels_1 = np.ones_like(x_data_1)
labels_2 = np.ones_like(x_data_2) * 2
labels_3 = np.ones_like(x_data_3) * 3
labels_4 = np.ones_like(x_data_4) * 4
labels_5 = np.ones_like(x_data_5) * 5
labels_6 = np.ones_like(x_data_6) * 6

# 全データを結合
x_data_combined = np.concatenate([x_data_1, x_data_2, x_data_3, x_data_4, x_data_5, x_data_6])
y_data_combined = np.concatenate([y_data_1, y_data_2, y_data_3, y_data_4, y_data_5, y_data_6])
labels_combined = np.concatenate([labels_1, labels_2, labels_3, labels_4, labels_5, labels_6])

# PyTorchテンソルに変換
x_data_combined = torch.tensor(x_data_combined, dtype=torch.float32)
y_data_combined = torch.tensor(y_data_combined, dtype=torch.float32)
labels_combined = torch.tensor(labels_combined, dtype=torch.float32)

In [4]:
class KnightShiftModel(nn.Module):
    def __init__(self):
        super(KnightShiftModel, self).__init__()
        # パラメータを定義し、requires_grad=Trueを指定
        self.k1 = nn.Parameter(torch.tensor(0.0, requires_grad=True))
        self.k2 = nn.Parameter(torch.tensor(0.0, requires_grad=True))
        self.k3 = nn.Parameter(torch.tensor(0.0, requires_grad=True))
        self.k4 = nn.Parameter(torch.tensor(0.0, requires_grad=True))
        self.k5 = nn.Parameter(torch.tensor(0.0, requires_grad=True))
        self.k6 = nn.Parameter(torch.tensor(0.0, requires_grad=True))
        self.t1 = nn.Parameter(torch.tensor(45.0, requires_grad=True))
        self.p1 = nn.Parameter(torch.tensor(45.0, requires_grad=True))
        self.t2 = nn.Parameter(torch.tensor(45.0, requires_grad=True))
        self.p2 = nn.Parameter(torch.tensor(45.0, requires_grad=True))

    def rotation_matrix(self, n, theta):
        K = torch.tensor([[0.0, -n[2], n[1]], [n[2], 0.0, -n[0]], [-n[1], n[0], 0.0]])
        I = torch.eye(3)
        theta = torch.tensor(theta)  # thetaをテンソルに変換
        R = I + torch.sin(theta) * K + (1.0 - torch.cos(theta)) * torch.matmul(K, K)
        return R

    def magnetic(self, axis, vec, theta):
        axis = torch.tensor(axis)
        vec = torch.tensor(vec)
        cos_theta = torch.cos(theta)
        sin_theta = torch.sin(theta)
        return (vec * cos_theta + 
                torch.cross(axis, vec) * sin_theta + 
                axis * torch.dot(axis, vec) * (1 - cos_theta))

    def forward(self, x, label):
        nx = torch.cos(torch.deg2rad(self.p1)) * torch.sin(torch.deg2rad(self.t1))
        ny = torch.sin(torch.deg2rad(self.p1)) * torch.sin(torch.deg2rad(self.t1))
        nz = torch.cos(torch.deg2rad(self.t1))
        sx = torch.cos(torch.deg2rad(self.p2)) * torch.sin(torch.deg2rad(self.t2))
        sy = torch.sin(torch.deg2rad(self.p2)) * torch.sin(torch.deg2rad(self.t2))
        sz = torch.cos(torch.deg2rad(self.t2))
        E = torch.eye(3)
        C3 = self.rotation_matrix([0,0,1], 2.0 * np.pi / 3.0)
        C3i = C3.T
        Rs = [E, C3i, C3]
        theta = torch.deg2rad(x)
        nt = torch.cross(torch.tensor([nx, ny, nz]), torch.tensor([sx, sy, sz]))
        h = self.magnetic([nx, ny, nz], [sx, sy, sz], theta)
        ht = self.magnetic(nt, [sx, sy, sz], theta)
        K = torch.tensor([[self.k1, self.k6, self.k5], [self.k6, self.k2, self.k4], [self.k5, self.k4, self.k3]])
        Ks = [torch.matmul(torch.matmul(R, K), R.T) for R in Rs]

        if label == 1:
            K1 = torch.matmul(h.T, torch.matmul(Ks[0], h))
            return K1
        elif label == 2:
            K2 = torch.matmul(h.T, torch.matmul(Ks[1], h))
            return K2
        elif label == 3:
            K3 = torch.matmul(h.T, torch.matmul(Ks[2], h))
            return K3
        elif label == 4:
            K4 = torch.matmul(ht.T, torch.matmul(Ks[0], ht))
            return K4
        elif label == 5:
            K5 = torch.matmul(ht.T, torch.matmul(Ks[1], ht))
            return K5
        elif label == 6:
            K6 = torch.matmul(ht.T, torch.matmul(Ks[2], ht))
            return K6

In [5]:
# モデル、損失関数、オプティマイザの設定
model = KnightShiftModel()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# トレーニングループ
for epoch in range(1000):
    optimizer.zero_grad()
    outputs = torch.zeros_like(y_data_combined)
    for i, (x, label) in enumerate(zip(x_data_combined, labels_combined)):
        outputs[i] = model(x, label)
    loss = criterion(outputs, y_data_combined)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/1000], Loss: {loss.item():.4f}')

# フィッティング結果のプロット
plt.figure(figsize=(10, 6))

# 各データセットのプロット
plt.plot(x_data_1, y_data_1, 'o', label='Data 1')
plt.plot(x_data_2, y_data_2, 'x', label='Data 2')
plt.plot(x_data_3, y_data_3, 's', label='Data 3')
plt.plot(x_data_4, y_data_4, 'h', label='Data 4')
plt.plot(x_data_5, y_data_5, 'd', label='Data 5')
plt.plot(x_data_6, y_data_6, 'p', label='Data 6')

# フィッティング結果のプロット
x_all = np.linspace(min(x_data_combined), max(x_data_combined), 100)
for label in [1, 2, 3, 4, 5, 6]:
    y_fit = np.array([model(torch.tensor(x, dtype=torch.float32), torch.tensor(label, dtype=torch.float32)).detach().numpy() for x in x_all])
    plt.plot(x_all, y_fit, label=f'Fit Label {label}')

plt.xlabel('Angle')
plt.ylabel('Knight shift')
plt.title('Fitting Results')
plt.legend()
plt.grid(True)

plt.show()

/var/folders/qy/8lm3fm3s3lg04nxt5ngs32zh0000gn/T/ipykernel_3660/1406042048.py:44: UserWarning: Using torch.cross without specifying the dim arg is deprecated.
Please either pass the dim explicitly or simply use torch.linalg.cross.
The default value of dim will change to agree with that of linalg.cross in a future release. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/Cross.cpp:66.)
  nt = torch.cross(torch.tensor([nx, ny, nz]), torch.tensor([sx, sy, sz]))
/var/folders/qy/8lm3fm3s3lg04nxt5ngs32zh0000gn/T/ipykernel_3660/1406042048.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  axis = torch.tensor(axis)
/var/folders/qy/8lm3fm3s3lg04nxt5ngs32zh0000gn/T/ipykernel_3660/1406042048.py:51: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and i

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn